In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import re

In [2]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

#url of the website
URL = 'https://myneta.info/WestBengal2021/index.php?action=summary&subAction=candidates_analyzed&sort=candidate#summary'

driver.get(URL)


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\sugan\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [3]:
#html parser for beautiful soup 
html = driver.page_source
soup = BeautifulSoup(html)

In [4]:
#creating a file 
file = open('bengal91.csv', 'w',newline='', encoding='utf8')
writer = csv.writer(file)
 
    
# write title row
writer.writerow(['sno','candidate','constituency','party','criminal_cases','education', 'total_assets','liabilities'])

84

In [5]:
results = soup.find_all(class_='divTableWithFloatingHeader')
results = results[1]
table_data = results.find_all("tr")

In [6]:


rows = []

for row in table_data[4:len(table_data)]:
    rows.append([val.text for val in row.find_all('td')])
    

writer.writerows(row for row in rows if row)
file.close()    

In [41]:
import pandas as pd

df = pd.read_csv('bengal91.csv')

In [42]:
#deleting unwanted columns
df = df.drop(['sno'], axis = 1) 

In [43]:
df.head()

,candidate,constituency,party,criminal_cases,education,total_assets,liabilities
0,Abdul Hai Mallik,ONDA,IND,0,8th Pass,"Rs 1,60,171 ~ 1 Lacs+",Rs 0 ~
1,Abdur Razzak Molla,FALTA,INC,2,10th Pass,"Rs 26,50,450 ~ 26 Lacs+",Rs 0 ~
2,Abhijit Bhattacharya,PURULIA,IND,0,Post Graduate,"Rs 44,39,532 ~ 44 Lacs+","Rs 3,75,000 ~ 3 Lacs+"
3,Abir Chandra Mandal,CHHATNA,IND,0,Graduate Professional,"Rs 8,09,000 ~ 8 Lacs+",Rs 0 ~
4,Adhikari Suvendu,NANDIGRAM,BJP,1,Post Graduate,"Rs 1,05,52,749 ~ 1 Crore+",Rs 0 ~


In [44]:
new = df["total_assets"].str.split('~', n = 1, expand = True)  #splitting the value
df = df.drop(['total_assets'], axis = 1)
df["total_assets"]= new[0] 



In [45]:
new1 = df["liabilities"].str.split('~', n = 1, expand = True)  #splitting the value
df = df.drop(['liabilities'], axis = 1) 
df["liabilities"]= new1[0] 



In [46]:
df['liabilities'] = df['liabilities'].replace({'Rs': '','Nil': 0 , ',':''}, regex=True)
df['total_assets'] = df['total_assets'].replace({'Rs': '','Nil': 0 , ',':''}, regex=True)

In [47]:
df.head()

,candidate,constituency,party,criminal_cases,education,total_assets,liabilities
0,Abdul Hai Mallik,ONDA,IND,0,8th Pass,160171,0
1,Abdur Razzak Molla,FALTA,INC,2,10th Pass,2650450,0
2,Abhijit Bhattacharya,PURULIA,IND,0,Post Graduate,4439532,375000
3,Abir Chandra Mandal,CHHATNA,IND,0,Graduate Professional,809000,0
4,Adhikari Suvendu,NANDIGRAM,BJP,1,Post Graduate,10552749,0


In [48]:
df['total_assets'] = df['total_assets'].astype(int)
df['liabilities'] = df['liabilities'].astype(int)

In [49]:
df

,candidate,constituency,party,criminal_cases,education,total_assets,liabilities
0,Abdul Hai Mallik,ONDA,IND,0,8th Pass,160171,0
1,Abdur Razzak Molla,FALTA,INC,2,10th Pass,2650450,0
2,Abhijit Bhattacharya,PURULIA,IND,0,Post Graduate,4439532,375000
3,Abir Chandra Mandal,CHHATNA,IND,0,Graduate Professional,809000,0
4,Adhikari Suvendu,NANDIGRAM,BJP,1,Post Graduate,10552749,0
...,...,...,...,...,...,...,...
562,Uttam Naskar,BISHNUPUR (SC),IND,0,10th Pass,511291,0
563,Uttara Singha Hazra,GARBETA,AITC,1,Graduate,17710285,6348523
564,Vibekananda Mondal,ULUBERIA UTTAR (SC),IND,0,12th Pass,11953,0
565,Vivekananda Bauri,RAGHUNATHPUR (SC),BJP,0,Graduate Professional,2030856,0


In [50]:
df.to_csv('west_bengal.csv', index=False)